<table>
<tr>
<td><img src="https://raw.githubusercontent.com/RafaelCaballero/APD/refs/heads/main/img/logoAPD.png" width="150"></td>
<td><table><tr><td><h1>Carpetas y ficheros</h1></td></tr>
           <tr><td><h3>Rafael Caballero Roldán</h3></td></tr></table></td>
<td><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTsPjCdm67xYS9AM7-dXQ46O23vaexAhnVJaQ&s" width="105"></td>
</tr>
</table>




### Índice
[Caminos](#Caminos)<br>
[Creación de carpetas](#Creación_de_carpetas)<br>
[Borrado](#Borrado)<br>
[Copia de ficheros](#Copia)<br>
[Ficheros .zip](#Zip)<br>

## Preparación

Ejecutar las dos siguientes celdas para asegurar que las bibliotecas que vamosa  usar están cargadas, y también para crear un par de ficheros de prueba

In [ ]:
modules = ["pathlib", "shutil", "pandas", "numpy","pyarrow"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")  
          try:        
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")
              
  print("¡Terminado!")

instala(modules) 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
path = Path.cwd() 

# datos
numcolumnas = 100
filas = []
numfilas = 10000
for i in range(numfilas):
    filas.append(np.random.randint(1,5000,numcolumnas))

df = pd.DataFrame(filas, columns = ["c"+str(i) for i in range(numcolumnas)])
fichero_csv = path / "prueba.csv"
fichero_parquet = path / "prueba.parquet"
df.to_csv(fichero_csv,index=False)
df.to_parquet(fichero_parquet, index=False)

<a name="Caminos"></a>
#### Caminos

En cualquier sistema operativo los sistemas de almacenamiento permanentes (discusos, memorias flash, etc) van a tener un sistema de carpetas y ficheros:
    
    * Las carpetas pueden contener ficheros y otras carpetas
    * Los ficheros contienen datos


El camino a un fichero se puede dar de dos formas:

#### Camino Absoluto

Ejemplo  **d:\docencia\2526\apd\codigo\path.ipynb**

Estos caminos comienzan generalmente por el nombre de unidad, luego la secuencia de carpetas y finalmente el nombre del fichero con su extensión. 

En el caso de Windows el separador de elementos es \ y en el caso de linux es /. Como la \ es un símbolo especial suele dar problemas, por eso Python incluye el prefijo r para poder usarlo:

        camino = r"c:\bertoldo\ficheros\datos.csv"

#### Camino Relativo

Se mueve desde el lugar en el que estamos (en el caso de un notebook sería desde el lugar donde está el notebook). Se utilia
- . Para indicar la carpeta actual como  **.\img\log.png**: que significa: busca dentro de la carpeta actual (.) una carpeta img y dentro de img el fichero log.png
- .. Para indicar el antecesor de la carpetaa anterior. Por ejemplo **..\datos\trafico.csv**, que indica que la carpeta datos es "hermana" de la actual

    
Debemos ser capaces de manejar estas estructuras en Python para automatizar nuestro proceso de adquisición de datos.

Primero veamos en qué carpeta nos encontramos ahora mismo (la del notebook de Python)

In [ ]:
from pathlib import Path
path = Path.cwd()  
print(path)

Añadimos un fichero al final del camino

In [ ]:
import pandas as pd
df = pd.DataFrame({"nombre":["Bertoldo", "Herminia","Calixta","Aniceto"],"edad":[18,19,24,30]})
p =  path / "datos.csv"             # Alternativa: Path(path,"datos.csv")
df.to_csv(p,index=False)
print(p)

¿El path existe? ¿es una carpeta o un fichero?

In [ ]:
if p.exists():
    print(p,"existe",end=" ")
    if p.is_dir():
        print("y es una carpeta")
    else:
        if p.is_file():
            print("y es un fichero")
else:
    print(p,"no existe")

Ya sabemos que p es un camino a un fichero, ahora vamos a ver algunas de sus características

In [ ]:
p.absolute

In [ ]:
p.drive

In [ ]:
p.parent

In [ ]:
p.name

In [ ]:
p.stem

In [ ]:
p.suffix

Además podemos obtener [algunos datos](https://docs.python.org/3/library/os.html#os.stat_result) del fichero, por ejemplo su tamaño en bytes

In [ ]:
estad = p.stat() # estadísticas varias
estad

In [ ]:
estad.st_size 

¿Y si no sé cómo se llama el fichero?

Todas las carpetas y ficheros del path actual (que debe ser una carpeta)

In [ ]:
def lista(path):
    print(path)
    espacio = " "*5
    for p in path.iterdir():
        if p.is_dir():
            print(espacio,p.name,": carpeta",sep="")
        else:
            if p.is_file():
                print(espacio,p.name,": fichero",sep="")
    print("="*40)
                
lista(path)                

También se puede usar `glob` que tiene alguna ventaja:

- Permite obtener solo los ficheros/carpetas que cumplan un patrón
- Tiene una versión recursiva que busca también en los hijos (ojo, esto puede hacer que sea lento si se hace en un sistema de ficheros grande)

In [ ]:
from pathlib import Path

root = Path.cwd()

# todos los ficheros de la carpeta actual
files = [p for p in root.glob("*") if p.is_file()]

# todos los ficheros de la carpeta actual y descendientes (recursivo)
rfiles = [p for p in root.rglob("*") if p.is_file()]

# solo CSV 
csvs = list(root.glob("*.csv"))

print(f"Esta carpeta tiene ficheros{len(files)} ficheros ({len(rfiles)} si contamos descendientes), de los cuales {len(csvs)} son csvs")

<a name="Creación_de_carpetas"></a>
#### Creación de carpetas

Es un buen hábito tener los datos iniciarles en una carpeta `raw` y dejar los resultados en una carpeta `preprocess`. Para ello debemos ser capaces de crear carpetas si no existen

In [ ]:
from pathlib import Path
path = Path.cwd()  
pathraw = Path(path,"raw")
if pathraw.exists():
    print("Ya existe")
else:
    print("No existe, lo creamos")
    pathraw.mkdir()
    if pathraw.exists():
        print("Creado")
    else:
        print("No se puede crear!!")

Si ya existe y no lo comprobamos  da error

In [ ]:
#pathraw.mkdir() Si quitamos el comentario

Pero se puede hacer mejor con el atributo exists_ok, que indica que si ya existe no haga nada (y si no lo crea)

In [ ]:
pathraw.mkdir(exist_ok=True)

Otro parámetro util para `mkdir` es parents=True, que creará además todos las carpetas intermedias. Esto es muy útil para crear una estructura compleja de ficheros

In [ ]:
folders = ["raw/download/feb","raw/download/mar","raw/download/may"] 
for f in folders:
    newFolder = Path(path,f)
    newFolder.mkdir(exist_ok=True,parents=True)

<a name="Borrado"></a>
#### Borrado de carpetas y ficheros

En principio disponemos de dos métodos interesantes   
    
    * path.unlink() borra un fichero
    * path.rmdir() borra una carpeta

In [ ]:
from pathlib import Path
path = Path.cwd() 
print("Antes")
lista(path)
fichero_csv = Path(path,"prueba.csv")
fichero_csv.unlink(missing_ok=True) # no da error aunque no exista
print("Después")
lista(path)

En el caso de borrar una carpeta debe estar vacía:

In [ ]:
from pathlib import Path
path = Path.cwd() 
raw_dir = path / "raw"
#raw_dir.rmdir() si quitamos el comentario dará error

Si no queremos borrar elemento a elemento y queremos borrar todo un subárbol podemos usar la librería `shutil`

OJO: esto puede borrar el disco duro entero sin preguntar, usar con precaución

In [ ]:
from shutil import rmtree
rmtree(raw_dir,ignore_errors=True) # para que no dé error si no existe

<a name="Copia"></a>
#### Copia de ficheros
Esta biblioteca también nos permite copiar ficheros de un destino a otro

Primero creamos algunas carpetas y un fichero de prueba 

In [ ]:
folders = ["raw/download/feb","raw/download/mar","raw/download/may"] 
for f in folders:
    newFolder = path / f
    newFolder.mkdir(exist_ok=True,parents=True)
    
import pandas as pd
df = pd.DataFrame({"nombre":["Bertoldo", "Herminia","Calixta","Aniceto"],"edad":[18,19,24,30]})
p = Path(folders[0]) / "datos.csv"
df.to_csv(p,index=False)
lista(Path(folders[0]))  

In [ ]:
import pathlib
import shutil

print("Antes")
lista(Path(folders[1]))
origen=p
destino= Path(folders[1]) / "datoscopia.csv"
shutil.copy(origen, destino)
print("después")
lista(Path(folders[1]))


In [ ]:
from shutil import rmtree
rmtree(raw_dir,ignore_errors=True) 

<a name="Zip"></a>
#### Ficheros .zip


Primero creamos algunas carpetas y un fichero de prueba 

In [ ]:
from pathlib import Path
path = Path.cwd() 
download_dir = Path(path,"raw/download")
download_dir.mkdir(exist_ok=True,parents=True)

fichero = "meteo22.zip"
path_fich = Path(download_dir,fichero)

In [ ]:
import requests
url = "https://github.com/RafaelCaballero/tdm/blob/master/datos/madrid/meteo22.zip?raw=true"
r = requests.get(url)
if r.status_code==200:
    with open(path_fich, 'wb') as f:
        f.write(r.content) # ahora lo grabamos localmente
    print("Grabado")
else:
    print("Error ",r.status_code)
    
lista(download_dir)    


Utilizamos `unpack_archive` que detecta el tipo de compresión y descomprime automáticamente. El primer parámetro es el fichero y el segundo la carpeta donde debe guardarse

In [ ]:
import shutil
raw_dir = Path(path,"raw")
shutil.unpack_archive(path_fich, raw_dir)
lista(raw_dir)

Para acabar un video corto con algunas ideas sobre nombres y organización de ficheros [https://www.youtube.com/watch?v=YslfY4W-NAg](https://www.youtube.com/watch?v=YslfY4W-NAg)